In [1]:
import sys
import neal
import greedy
import tabu
import time
import numpy as np

from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../code/'))
from experiment import Experiment
from table import Table
from visualisation import Figure

# Load the data

In [2]:
data_folder = '../data/Quadratic Assignment Problem/'

file_names = ['had12.npz', 'had14.npz', 'had16.npz',
             'had18.npz', 'had20.npz', 'rou12.npz',
             'rou15.npz', 'rou20.npz', 'tai40a.npz',
             'tai40b.npz']

loaded_files = [np.load(data_folder + file) for file in file_names]

obj_qubos = [i['cost_function_qubo'] for i in loaded_files]
obj_constants = [i['cost_function_constant'] for i in loaded_files]
con_qubos = [i['constraint_function_qubo'] for i in loaded_files]
con_constants = [i['constraint_function_constant'] for i in loaded_files]

# Prepare the data

Here we already get the qubos and constraints. So we only need to calculate penalties and get the full QUBOs.

In [3]:
minimisation = True # This is a minimisation problem
QUBOs, penalties = Experiment.data_prep_light(obj_qubos, con_qubos, 'Verma&Lewis', minimisation)
qubo_sizes = [max(qubo, key=tuple)[0] + 1 for qubo in QUBOs]

# Run experiments

In [4]:
# The number of samples we want the sampler to return
repeats = 60

## Greedy

In [5]:
greedy_sampler = greedy.SteepestDescentSampler()
greedy_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                     greedy_sampler, repeats, num_reads=150)

100.0 %


## Simulated Annealing

In [6]:
sa_sampler = neal.SimulatedAnnealingSampler()
sa_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                 sa_sampler, repeats, num_reads=30)

100.0 %


## Tabu

In [7]:
tabu_sampler = tabu.TabuSampler()
tabu_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                   tabu_sampler, repeats, timeout=2700)

100.0 %


# Record the results

In [8]:
greedy_results = Table.record_results(greedy_runs, qubo_sizes, penalties, repeats, minimisation)
sa_results = Table.record_results(sa_runs, qubo_sizes, penalties, repeats, minimisation)
tabu_results = Table.record_results(tabu_runs, qubo_sizes, penalties, repeats, minimisation)

# Display the first repetition table
rep = 0
Table.display_side_by_side(greedy_results[rep], sa_results[rep], tabu_results[rep], titles=['Greedy', 'SA', 'Tabu'])

,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,144,10920,1756,0,1756
1,196,17936,2900,0,2900
2,256,25160,3892,0,3892
3,324,32204,5566,0,5566
4,400,41856,7290,0,7290
5,144,1749888,256048,0,256048
6,225,2996352,386990,0,386990
7,400,5138348,804246,0,804246
8,1600,20837608,3448552,0,3448552
9,1600,9048288550,756679343,0,756679343


# Explore the results

In [9]:
# Show total energies of all tries in all problems in a single df
energies_greedy = Table.columns_to_table(greedy_results, 'Energy (minimisation)')
energies_sa = Table.columns_to_table(sa_results, 'Energy (minimisation)')
energies_tabu = Table.columns_to_table(tabu_results, 'Energy (minimisation)')

energies_tabu

,Energy (minimisation) 0,Energy (minimisation) 1,Energy (minimisation) 2,Energy (minimisation) 3,Energy (minimisation) 4,Energy (minimisation) 5,Energy (minimisation) 6,Energy (minimisation) 7,Energy (minimisation) 8,Energy (minimisation) 9,...,Energy (minimisation) 50,Energy (minimisation) 51,Energy (minimisation) 52,Energy (minimisation) 53,Energy (minimisation) 54,Energy (minimisation) 55,Energy (minimisation) 56,Energy (minimisation) 57,Energy (minimisation) 58,Energy (minimisation) 59
0,1684,1678,1680,1678,1680,1670,1678,1672,1674,1682,...,1686,1674,1684,1678,1674,1678,1672,1686,1676,1686
1,2754,2778,2782,2760,2762,2776,2754,2784,2768,2752,...,2782,2766,2770,2778,2750,2768,2770,2762,2748,2752
2,3776,3810,3774,3774,3792,3784,3786,3804,3794,3802,...,3798,3780,3796,3792,3786,3776,3784,3800,3768,3794
3,5442,5454,5440,5452,5450,5432,5488,5472,5480,5476,...,5422,5470,5442,5462,5460,5476,5456,5448,5468,5446
4,7042,7070,7058,7032,7120,7070,7096,7052,7036,7046,...,7082,6980,7064,7058,7052,7018,7076,7050,7092,7088
5,241122,242632,240906,245010,235852,243582,238134,242102,244668,241706,...,242102,244044,235654,243498,242102,241706,243606,240630,242102,241484
6,376638,371758,370156,377792,367522,371480,373364,379618,370580,372834,...,365804,365804,369834,375680,369432,379800,370978,376252,373702,372842
7,759002,777338,777738,769860,764554,765722,775026,761120,783290,764508,...,781100,761890,762584,766806,774816,768082,763832,779406,774220,776248
8,3342566,3345002,3346892,3343596,3334578,3338404,3361894,3351826,3338666,3326188,...,3320078,3352426,3343092,3323448,3322528,3326188,3354874,3356980,3362758,3360874
9,687145351,657960154,712455404,681504037,709751544,707883003,692563388,695127731,703934641,696264411,...,669407636,691410266,697032605,707542650,707574384,693637270,701818289,698277958,696263138,696506204


In [10]:
# Show number of broken constraints of all tries in all problems in a single df
broken_constraints_greedy = Table.columns_to_table(greedy_results, 'Broken Constraints')
broken_constraints_sa = Table.columns_to_table(sa_results, 'Broken Constraints')
broken_constraints_tabu = Table.columns_to_table(tabu_results, 'Broken Constraints')

broken_constraints_greedy

,Broken Constraints 0,Broken Constraints 1,Broken Constraints 2,Broken Constraints 3,Broken Constraints 4,Broken Constraints 5,Broken Constraints 6,Broken Constraints 7,Broken Constraints 8,Broken Constraints 9,...,Broken Constraints 50,Broken Constraints 51,Broken Constraints 52,Broken Constraints 53,Broken Constraints 54,Broken Constraints 55,Broken Constraints 56,Broken Constraints 57,Broken Constraints 58,Broken Constraints 59
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Show feasible solutions
feasible_full_greedy = Table.feasibility_table(greedy_results)
feasible_full_sa = Table.feasibility_table(sa_results)
feasible_full_tabu = Table.feasibility_table(tabu_results)

feasible_full_tabu

,Feasible 0,Feasible 1,Feasible 2,Feasible 3,Feasible 4,Feasible 5,Feasible 6,Feasible 7,Feasible 8,Feasible 9,...,Feasible 50,Feasible 51,Feasible 52,Feasible 53,Feasible 54,Feasible 55,Feasible 56,Feasible 57,Feasible 58,Feasible 59
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


This is the final and the most important table. It displays the number of feasible solutions achieved for every problem, the feasibility rate (feasible_soultions/total_solutions), the mean energy of the solutions and the standard deviation.

In [12]:
# Calculate number of feasible solutions with mean and SD (in all runs)
feasible_greedy = Table.feasibility_statistic(greedy_results)
feasible_sa = Table.feasibility_statistic(sa_results)
feasible_tabu = Table.feasibility_statistic(tabu_results)

# Display the table
Table.display_side_by_side(feasible_greedy, feasible_sa, feasible_tabu, titles=['Greedy', 'SA', 'Tabu'])

,Feasible,Feasibility rate,Energy mean,Energy SD
0,60.0,1.0,1.739267e+03,2.069952e+01
1,60.0,1.0,2.870967e+03,3.238014e+01
2,60.0,1.0,3.892900e+03,3.268841e+01
3,60.0,1.0,5.606733e+03,3.325294e+01
4,60.0,1.0,7.286133e+03,4.208767e+01
5,60.0,1.0,2.571876e+05,3.500843e+03
6,60.0,1.0,3.941395e+05,5.232060e+03
7,60.0,1.0,8.054516e+05,7.180451e+03
8,60.0,1.0,3.453470e+06,1.505934e+04
9,60.0,1.0,7.573762e+08,1.598683e+07


# Save results

In [13]:
data_folder = '../Data/Produced/Quadratic Assignment Problem/'
broken_constraints_greedy.to_pickle(data_folder + 'vlewis_qap_greedy_broken_constraints.pkl')
broken_constraints_sa.to_pickle(data_folder + 'vlewis_qap_sa_broken_constraints.pkl')
broken_constraints_tabu.to_pickle(data_folder + 'vlewis_qap_tabu_broken_constraints.pkl')

feasible_greedy.to_pickle(data_folder + 'vlewis_qap_greedy_feasible.pkl')
feasible_sa.to_pickle(data_folder + 'vlewis_qap_sa_feasible.pkl')
feasible_tabu.to_pickle(data_folder + 'vlewis_qap_tabu_feasible.pkl')